# Advent of Code Problems

### Key Concepts & Packages

The following packages and ideas are part of the python language itself. They are always available and understanding them will make your life much easier. Individually, they help solve simple problems in the most efficient way, allowing you to combine them to solve more complex problems

  * [Truthy and Falsy values](https://www.freecodecamp.org/news/truthy-and-falsy-values-in-python/)
  * [Indexing](https://towardsdatascience.com/the-basics-of-indexing-and-slicing-python-lists-2d12c90a94cf) and [numpy indexing](https://numpy.org/doc/stable/reference/arrays.indexing.html)
  * Regular expressions [re](https://docs.python.org/3/library/re.html)
  * Hashes and [Collections](https://docs.python.org/3/library/collections.html)


These modules aren't part of the core python language, but they're well written and very widely used.

* [Requests](https://docs.python-requests.org/en/master/) HTTP for humans
* [Pandas](https://pandas.pydata.org/)
* [Numpy](https://numpy.org/)

There are **lots** of other python modules available as well. In general you should look for an existing python module before writing something from scratch, it'll save you a lot of work and let you focus on the interesting parts of your problem. [PyPI](https://pypi.org/) is the canonical source for many packages and integrates with the `pip` package manager. When you are looking at a new package, some of the things to consider are

  1. Does it do what you need? Does it do it well?
  1. Does it have a good support community and active development? Often the associated GitHub issue queue can help you judge this
  
  


## Problem 1: Password Checking

### Part 1

 * N.B. This problem is a tweak of [Advent of Code 2020 problem 2](https://adventofcode.com/2020/day/2)
You've been given the password file for a group of users and you've been asked to validate the entries. Here are some sample entries

```
1-9 a: asjuaycbnwo
3-7 c: qohnfbyvdvxb
5-7 j: abnkjahajjay
3-6 p: papvmnnapa
1-5 n: nannanuuanyn
```

The second field with the colon represents a character that we expect to find in the password (the 3rd field). The first field represents the minimum and maximum number of occurrences of that character which are needed to satisfy our policy. For example, the first line says that the password is `asjuaycbnwo` and that we expect to find at least one to at most nine occurrences of the latter `a` in that password. Given those rules we have

```
1-9 a: asjuaycbnwo   # VALID   2 occurences of "a" (1 <= 2 <= 9)
3-7 c: qohnfbyvdvxb  # INVALID 0 occurences of "c" (0 < 3)
5-7 j: abnkhajajjay  # INVALID 3 occurences of "j" (3 < 5)
3-6 p: papvmnnapa    # VALID   3 occurences of "p" (3<= 3 <= 6)
1-5 n: nannnnuuanyn  # INVALID 7 occurences of "n" (7 > 5)
```

You are asked to calculate the total number of valid passwords in the file.

In [3]:
data_url = "https://m2pi.syzygy.ca/data/day_2.txt"

#### Pure Python Version

In [37]:
import re
import requests

try:
    r = requests.get('https://m2pi.syzygy.ca/data/day_2.txt')
    r.raise_for_status()
except Exception as err:
    print(f'Unable to retrieve passwords: {err}')

lines = r.text.strip().split('\n')
lines[:10]

['15-19 k: kkkkkkkkkkkkzkkkkkkk',
 '1-11 s: sbssswsqsssssrlss',
 '8-9 b: pbbbbbbkbz',
 '4-10 w: wwccwcqwdmbktjrxhw',
 '1-6 x: jvscgqsnt',
 '1-7 x: xxxxxxcx',
 '6-10 s: smssssfskssdwvtcss',
 '6-12 q: qqqqzqqjqfqdqq',
 '3-7 d: ddwbzbf',
 '12-14 s: ssdssssssssmsq']

In [38]:
pwre = re.compile(
    '(?P<chmin>\d+)-(?P<chmax>\d+) (?P<ch>[a-z]): (?P<pw>[a-z]+)'
)

valid_pw = 0
for line in lines:
    m = pwre.match(line).groupdict()
    m['chmin'], m['chmax'] = int(m['chmin']), int(m['chmax'])
    occurrences = len(re.findall(m['ch'], m['pw']))
    if (m['chmin'] <= occurrences <= m['chmax']):
        valid_pw += 1

print(f"{valid_pw} VALID passwords ({len(lines) - valid_pw} INVALID)")

655 VALID passwords (345 INVALID)


#### Pandas Version

In [39]:
import pandas as pd

df = pd.read_csv(
    data_url,
    delimiter=' ',
    names=['letter_count', 'letter', 'password']
)
df['letter'] = df['letter'].str.replace(':','')
df[['letter_min', 'letter_max']] = df['letter_count'].str.split('-', expand=True).astype(int)
df.head()

,letter_count,letter,password,letter_min,letter_max
0,15-19,k,kkkkkkkkkkkkzkkkkkkk,15,19
1,1-11,s,sbssswsqsssssrlss,1,11
2,8-9,b,pbbbbbbkbz,8,9
3,4-10,w,wwccwcqwdmbktjrxhw,4,10
4,1-6,x,jvscgqsnt,1,6


In [40]:
def valid_password(password, c, min_c, max_c):
    return (min_c <= password.count(c)) and (password.count(c) <= max_c)

In [41]:
df.apply(lambda x: valid_password(x['password'], x['letter'], x['letter_min'], x['letter_max']), axis=1).sum()

655

### Part 2

Part two of this problem says that the schema was misinterpreted in part 1. Actually, the first token on each line should be interpretated as positions in the password string (indexed from 1!) and that the given character should occur **exactly** once in those positions. Recycling our examples from above

```
1-9 a: asjuaycbnwo  # VALID "a" occurs in position 1
3-7 c: qohnfbyvdvxb # INVALID "c" does not occur in position 3 or 7
5-7 j: abnkhajajjay # VALID "j" occurs in position 5
3-6 p: papvmnnapa   # VALID "p" occurs in position 3
1-5 n: nannnnuuanyn # INVALID "n" occurs in position 1 and position 5
```

#### Pure Python Version

In [43]:
lines = r.text.strip().split('\n')
lines[:10]
valid_pw = 0
for line in lines:
    chmin, chmax, ch, pw = pwre.match(line).groups()
    chmin, chmax = int(chmin), int(chmax)
    if (pw[chmin - 1] == ch) ^ (pw[chmax - 1] == ch):
        valid_pw += 1
        
print(f"{valid_pw} VALID passwords ({len(lines) - valid_pw} INVALID)")

673 VALID passwords (327 INVALID)


#### Pandas Version

In [44]:
def validate_password(pw, c, pos1, pos2):
    return (pw[pos1 - 1] == c) ^ (pw[pos2 - 1] == c)

In [45]:
df.apply(
    lambda x: validate_password(x.password, x.letter, x.letter_min, x.letter_max
), axis=1).sum()

673

**N.B. This is a trick question. The correct answer is that these passwords are all given in plain text which should never ever happen so they are all invalid!**

## Problem 2

In this problem, people are asked to answer yes or no to a list of 26 questions (labeled 'a', 'b', ... 'z'). The responses are recorded by noting down the label of each question for which the response was "yes". For example, responses for three people might look like
```
afuy
rz
aypq
```
The first person has answered yes to questions `a`, `f`, `u` and `y`. The second has answered yes to `r` and `z` etc. We assume that everyone answered yes to at least one question.

The responses of _groups_ of people will be considered with each group being separated by a single blank line. Extending our example from above...
```
afuy
rz
aypq

anqvz

bchklp
bnrt
```
Represents 3 groups. The second group has one member and the third has two members.


In [46]:
data_url = "https://m2pi.syzygy.ca/data/day_6.txt"

### Part 1

In the first part of the problem we are asked to count the number of questions answered "yes" by each group. Duplicate answers are only counted once e.g. For the first group the answer should be 8 (`a` and `y` appear twice but we only count them once.

### Part 1 Answer

Python collections are almost always going to be the foundation of your solution. In this case, sets do what we want to do (throw away duplicate entries).

 * Split entries into groups
 * Find answered questions without duplicates

In [ ]:
import re
import requests

try:
    r = requests.get('https://m2pi.syzygy.ca/data/day_6.txt')
    r.raise_for_status()
except Exception as err:
    print(f'Unable to retrieve passwords: {err}')

group_answers = r.text.strip().split('\n\n')
group_answers[:10]

In [53]:
valid_pw = 0
for line in lines:
    chmin, chmax, ch, pw = pwre.match(line).groups()
    chmin, chmax = int(chmin), int(chmax)
    if (pw[chmin - 1] == ch) ^ (pw[chmax - 1] == ch):
        valid_pw += 1
        
print(f"{valid_pw} VALID passwords ({len(lines) - valid_pw} INVALID)")

673 VALID passwords (327 INVALID)


### Part 2

In part 2 of this problem, we are told to count the total number of questions for which all participants in a group answered yes. For our example
above...
```
afuy
rz
aypq

anqvz

bchklp
bnrt
```
The total for the first group would be 0 (no letter appears in all three lines). The total for the second group would be 5 because the group size is 1 `a`, `n`, `q`, `v`, `z` all appear once. The total for the third group would be 1 since `b` is the only letter which appears on all lines.

### Part 2 Answer

The collections module in the python standard library has lots of fantastic features. For this problem the `Counter` object behaves like the `set` from above, but keeps track of exactly how many of each object have been seen before.

#### Pure Python Version

In [54]:
from collections import Counter

all_yes = 0
for answers in group_answers:
    all_yes += sum([x == len(answers.strip().split('\n')) for x in Counter(answers.replace('\n','')).values()])
    
print(f"{all_yes} questions where all group memebers answered YES")

3103 questions where all group memebers answered YES


##### Pandas Version

In [55]:
def count_yes(responses):
    return sum([x == len(responses.strip().split('\n')) for x in Counter(responses.replace('\n', '')).values()])

In [56]:
count_yes(answers)

10

In [57]:
df['everyone_yes'] = df.apply(lambda x: count_yes(x.answers), axis=1)
df['everyone_yes'].sum()

3103